In [ ]:
import sys, os

In [ ]:
import getpass

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

user = getpass.getuser()

if user == "sidsel":
    parquet_path = "/home/sidsel/workspace/sparkdata/parquet"
elif user == "svanhmic":
    parquet_path = "/home/svanhmic/workspace/data/DABAI/sparkdata/parquet"

In [ ]:
#from sample.DataIO import DataIO
#data = DataIO(sc,feature_path=parquet_path+"/featureDataCvr.parquet",company_path=parquet_path+"/companyCvrData")
#feature_data = data.mergeCompanyFeatureData()

In [ ]:
#import data!
 
from shared.create_dummy_data import DummyData 

dummy_data = DummyData(sc,1000)
dummy_data.create_outliers("x",10)
dummy_data.create_outliers("y",10)
dummy_data.show()

In [ ]:
#Select parameters
from sample.CreateParameters import CreateParameters


params = CreateParameters(dummy_data.df.columns)

In [ ]:
params.select_params()

In [ ]:
#print(params.export_values())
dicKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'label':('label'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'label':("cvrNummer","navn"), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}
dicBKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'BisectingKMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'label':('label'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'BisectingKMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'label':("cvrNummer","navn"), 'standardize': False, 'clusters': 2, 'tolerance': 0.0001}

# NOTE!!! Data is note standardized due to random variable is between 0-1 and outliers are beyond
dicGMM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'GaussianMixture', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'label':('label'), 'standardize': False, 'clusters': 10, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'GaussianMixture', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'label':("cvrNummer","navn"), 'standardize': True, 'clusters': 10, 'tolerance': 0.0001} #
#{'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'initialstep': 10, 'standardize': True, 'clusters': 3, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'Prediction', 'iterations': 10}

In [ ]:
from sample.ExecuteWorkflow import ExecuteWorkflow
from random import random

import numpy as np

execution_model = ExecuteWorkflow()
#print(execution_model.params)
execution_model.params = dicGMM#params.export_values()

In [ ]:
runTest = execution_model.run(data=dummy_data.df)

In [ ]:
from sample.ShowKmeans import ShowResults
results = ShowResults(execution_model.params)
results.select_prototypes(runTest)